In [1]:
from __future__ import annotations
from data_handler import *
from analysis import *
from dsp_premitives import *
from processors import *
from render_engine import *
from pre_master_prep import *
from streaming_normalization_simulator import *
from comparison_reporting import *
from presets_recommendations import *
from logging_versioning import *
from utils import *
from mastering_orchestrator import *

Post-Mix I/O layer loaded: AudioBuffer, load_wav, save_wav, resample_poly, slice_preview, with_suffix, auto_out_path, sha256_file, print_audio_summary.
Convenience helpers loaded: make_workspace, batch_load_wavs, env_fingerprint, Manifest, write/read_manifest, register_input, register_artifact, import_mix.
Analysis layer loaded: analyze_wav/analyze_audio_array, analysis_table, plot_spectrum, plot_short_term_loudness, plot_waveform_excerpt, LUFS approx, true-peak approx, stereo & health metrics.
DSP Primitives Layer loaded:
- Gain/level: apply_gain_db, normalize_peak, normalize_lufs, measure_peak, measure_rms
- Filters: highpass_filter, lowpass_filter, bandpass_filter, shelf_filter, peaking_eq, notch_filter, tilt_eq
- Stereo: mid_side_encode, mid_side_decode, stereo_widener
- Dynamics: compressor (soft‑knee), transient_shaper
- Fades: fade_in, fade_out
- K‑weighting/LUFS approx: k_weight, lufs_integrated_approx
Patched: render_from_cache now unpacks widen_stereo tuple correctly.
Process

## MAIN

In [4]:
# MAIN: end-to-end sanity test for I/O → Analysis → DSP Primitives → Processors
# Run this cell after loading the previous cells.
# It will:
#  - create a workspace
#  - import your mix (set MIX_SRC below)
#  - analyze the original
#  - build a preview cache
#  - render a few dial presets
#  - save outputs, register artifacts
#  - compare metrics in a table
#  - plot spectrum & loudness overlays
#
# Safe to re-run; it creates a new timestamped workspace each time.

import os, numpy as np, matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd


# ---------- 1) SET YOUR SOURCE FILE HERE ----------
MIX_SRC = "/Users/itay/Documents/muxing/ITAY - CRASHING v.6 MIX ONLY.wav"   # <-- change me
MIX_SRC = "/Users/itay/Documents/muxing/ITAY - 4 CHORDS v.21 STEFAN.wav"   # <-- change me
MIX_SRC = "/Users/itay/Documents/post_mix_data/input_mix_raw/ITAY - HOTEL POLYPHONIA STEFAN.wav"   # <-- change me



In [5]:
# ============================================
# MAIN — Full End-to-End Pipeline (premastered-only)
# ============================================

import os, json, numpy as np, soundfile as sf
from dataclasses import asdict

# ---------- 0) SET YOUR SOURCE FILE HERE ----------
PROJECT  = "postmix_v1"  # MIX_SRC must be set in a previous cell

# ---------- 1) Global toggles ----------
DO_STREAMING_SIM = False   # no mastering, no streaming sim
RECS_LIMIT       = 3
REPORT_REF_NAME  = "Original"

# ---------- 2) Workspace, manifest, logger, environment ----------
paths = make_workspace(
    base_dir=os.path.expanduser("/Users/itay/Documents/post_mix_data/PostMixRuns"),
    project=PROJECT
)
man   = Manifest(project=PROJECT, workspace=paths)

workspace_root = os.path.expanduser("/Users/itay/Documents/post_mix_data/PostMixRuns")
logger = RunLogger.start(workspace_root, tag=PROJECT)

env    = capture_environment()
logger.log_event("env", {"environment": env})

# bring input into workspace + register/log
if not os.path.exists(MIX_SRC):
    raise FileNotFoundError(f"Set MIX_SRC to a valid path, got: {MIX_SRC}")
mix_path = import_mix(paths, MIX_SRC, alias="mix.wav")
register_input(man, mix_path, alias="mix")
register_and_log_artifact(man, logger, mix_path, kind="input",
                          params={"alias":"mix"}, stage="import_mix")

# ---------- 3) Load, validate, analyze original ----------
x, sr = sf.read(mix_path)
ensure_audio_valid(x, "mix")

buf = load_wav(mix_path)
print_audio_summary(buf, "Original Mix")

rep_orig = analyze_wav(mix_path)
logger.log_metrics("analysis_original", {
    "sr": rep_orig.sr, "duration_s": rep_orig.duration_s,
    "peak_dbfs": rep_orig.basic["peak_dbfs"],
    "true_peak_dbfs": rep_orig.true_peak_dbfs,
    "rms_dbfs": rep_orig.basic["rms_dbfs"],
    "lufs_int": rep_orig.lufs_integrated,
    "crest_db": rep_orig.basic["crest_db"],
    "bass_%": rep_orig.bass_energy_pct,
    "air_%": rep_orig.air_energy_pct,
    "phase_corr": rep_orig.stereo["phase_correlation"],
    "stereo_width": rep_orig.stereo["stereo_width"],
    "spectral_flatness": rep_orig.spectral_flatness,
})

# ---------- 4) Recommendations (dials) ----------
recs = recommend_from_analysis(rep_orig)
print(recommendation_summary(recs))

variants_plan = build_premaster_plan_from_recs(recs, limit=RECS_LIMIT, prefix="PM")
variants_plan.insert(0, ("Transparent", DialState(bass=0, punch=0, clarity=0, air=0, width=0)))

logger.log_params("recommendations", {
    "plan": [(name, asdict(d)) for (name, d) in variants_plan]
}, code_versions={"presets_recs": CODE_VERSIONS["presets_recs"]})

# ---------- 5) RenderEngine: preprocess + premaster variants ----------
engine = RenderEngine(x, sr, preprocess=PreprocessConfig(
    low_cutoff=120.0, kick_lo=40.0, kick_hi=110.0
))
pre_meta = engine.preprocess()
logger.log_params("render_preprocess_cache", pre_meta,
                  code_versions={"processors": CODE_VERSIONS["processors"],
                                 "render_engine": CODE_VERSIONS["render_engine"]})

# Generate all premastered variants directly into paths.premastered
opts = RenderOptions(target_peak_dbfs=CFG.render_peak_target_dbfs,
                     bit_depth=CFG.default_bit_depth,
                     hpf_hz=None, save_headroom_first=False)
var_metas = engine.commit_variants(paths.premastered, variants_plan, opts=opts)

for meta in var_metas:
    register_and_log_artifact(man, logger, meta["out_path"], kind="premastered",
                              params=meta, stage=f"variant__{os.path.basename(meta['out_path'])}")

# ---------- 6) Reporting (metrics, plots, HTML) ----------
compare_files = [mix_path] + [m["out_path"] for m in var_metas]
compare_files = [p for p in compare_files if os.path.exists(p)]

cfg_cmp = CompareConfig(
    preview_seconds=CFG.preview_seconds,
    nfft=CFG.nfft,
    reference_name=REPORT_REF_NAME
)

reports_assets_dir = os.path.join(paths.reports, "assets")
os.makedirs(reports_assets_dir, exist_ok=True)

bundle = write_report_bundle(
    file_paths=compare_files,
    reports_dir=reports_assets_dir,
    cfg=cfg_cmp,
    manifest=man,
    report_name="comparison_report.html",
    extra_notes="Auto-generated end-to-end run."
)

try:
    html_enh = write_report_html_enhanced(
        bundle["summary_df"], bundle["deltas_df"], bundle["plots"],
        os.path.join(reports_assets_dir, "comparison_report_enhanced.html"),
        title="Post-Mix Comparison Report",
        extra_notes="Client review pack",
        code_versions=CODE_VERSIONS,
        dial_snapshot=variants_plan[1][1].__dict__ if len(variants_plan)>1 else {}
    )
    register_and_log_artifact(man, logger, html_enh, kind="report",
                              params={"enhanced": True},
                              stage="compare_html_enhanced")
except NameError:
    pass

# ---------- 7) Reproducibility bundle ----------
zip_path = os.path.join(paths.reports, "bundles", f"{logger.run_id}.zip")
repro_zip = make_repro_zip(
    zip_path,
    workspace_root=paths.root,
    run_logger=logger,
    env_info=env,
    inputs=[mix_path],
    outputs=[m["out_path"] for m in var_metas],
    extra_jsons={"code_versions": CODE_VERSIONS,
                 "recommendations": [(n, asdict(d)) for (n,d) in variants_plan]},
    readme_text="Bundle created by MAIN end-to-end run."
)
register_and_log_artifact(man, logger, repro_zip, kind="bundle",
                          params={"run_id": logger.run_id}, stage="repro_zip")

# ---------- 8) Finalize ----------
logger.write_summary({
    "project": PROJECT,
    "run_id": logger.run_id,
    "mix": mix_path,
    "premastered_variants": [m["out_path"] for m in var_metas],
    "report": bundle["html_path"],
    "env_sha": json_sha256(env),
})
write_manifest(man)

print("\n=== DONE ===")
print("Workspace:", paths.root)
print("Premastered variants:", *[m["out_path"] for m in var_metas], sep="\n - ")
print("Report:", bundle["html_path"])
print("Repro bundle:", repro_zip)


Workspace created at: /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_v1_20250820-191920
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_v1_20250820-191920/inputs/mix.wav
Original Mix: sr=44100 | ch=2 | dur=322.742s | peak=0.833367 | rms=0.101539
  path: /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_v1_20250820-191920/inputs/mix.wav
  sha256: 88cb9cf26c7530ec...
  src dtype: int32 | src ch: 2
- Punch Up the Transients (priority 2): B16 P38 C10 A14 W6
    • crest 18.2 dB (dynamic) and LUFS -21.7 (quiet) → add transient definition
    • small air lift to help intelligibility


KeyboardInterrupt: 